In [1]:
import ast
import json
import pandas as pd

In [2]:
df_category = pd.read_csv("../data/df_category_sim.csv").drop(columns=['Unnamed: 0'])
df_links = pd.read_csv("../data/df_links_sim.csv").drop(columns=['Unnamed: 0'])
df_lda = pd.read_csv("../data/df_lda_sims.csv").drop(columns=['Unnamed: 0'])

In [3]:
df_category.head()

,Title,Path,Jaccard
0,"2,5-Dimethylfuran","['Global_warming', 'Climate change', 'Politics...","[1.0, 1.0, 1.0, 0.7142857142857143, 0.14285714..."
1,2-Methylfuran,"['Global_warming', 'Climate change', 'Politics...","[1.0, 1.0, 1.0, 0.7142857142857143, 0.14285714..."
2,2007–08 world food price crisis,"['Global_warming', 'Climate change', 'Politics...","[1.0, 1.0, 1.0, 0.7142857142857143, 0.14285714..."
3,ASEAN Wildlife Enforcement Network,"['Global_warming', 'Climate change', 'Politics...","[0.7142857142857143, 0.7142857142857143, 0.714..."
4,Accuracy in Media,"['Global_warming', 'Climate change', 'Climate ...","[0.14285714285714285, 0.14285714285714285, 0.1..."


In [4]:
df_links.head()

,Title,Links,Jaccard
0,"2,5-Dimethylfuran","['2,5-Dimethylfuran (data page)', 'Aqueous sol...","[1.0, 0.24719101123595505, 0.01578947368421052..."
1,2-Methylfuran,"['2-Methylfuran (data page)', 'Alternative fue...","[0.24719101123595505, 1.0, 0.00295857988165680..."
2,2007–08 world food price crisis,"['2000s commodities boom', '2000s energy crisi...","[0.015789473684210527, 0.0029585798816568047, ..."
3,ASEAN Wildlife Enforcement Network,"['ASEAN', 'ASEAN Center for Biodiversity', 'As...","[0.0, 0.0, 0.007894736842105263, 1.0, 0.014150..."
4,Accuracy in Media,"['2012 Benghazi attack', 'Advocacy journalism'...","[0.009259259259259259, 0.0058823529411764705, ..."


In [5]:
df_lda.head()

,Title,Text_processed,Dominant Topic,Sims
0,"2,5-Dimethylfuran","2,5-dimethylfuran is a heterocyclic compound w...",40,"[1.0000000000000002, 0.6208810150204145, 0.029..."
1,2-Methylfuran,"2-methylfuran, also known with the older name ...",20,"[0.6208810150204145, 1.0000000000000024, 0.001..."
2,2007–08 world food price crisis,world food prices increased dramatically in 20...,39,"[0.02914683177495991, 0.0016481418417907804, 0..."
3,ASEAN Wildlife Enforcement Network,the asean wildlife enforcement network (asean-...,3,"[0.00047928602761282814, 0.002529903057432023,..."
4,Accuracy in Media,accuracy in media (aim) is an american non-pro...,21,"[0.0005204638786978396, 0.0024441338682554127,..."


In [6]:
df_category['Jaccard'] = df_category['Jaccard'].apply(lambda x: ast.literal_eval(x))
df_links['Jaccard'] = df_links['Jaccard'].apply(lambda x: ast.literal_eval(x))
df_lda['Sims'] = df_lda['Sims'].apply(lambda x: ast.literal_eval(x))

In [7]:
df_category['Jaccard'].iloc[0][0]

1.0

In [16]:
data_stories = ['Methane', 'Global methane initiative', 'Methane emissions', 'Fugitive gas emissions',
                'Hydraulic fracturing', 'Environmental impact of hydraulic fracturing', 'Renewable natural gas', 
                'Biogas', 'Biofuel', 'Climate change denial', 'Fossil fuels lobby', 'Politics of global warming', 
                'Wood gas', 'Afforestation', 'Reforestation', 'Reducing emissions from deforestation and forest degradation',
                'Pollution prevention', 'Politics of global warming', 'Green economy', 'Environmental economics', 'Charcoal', 
                'Groundwater recharge', 'Green New Deal']

In [17]:
causes = ['Methane emissions', 'Hydraulic fracturing', 'Media coverage of global warming', 'Fossil fuels lobby']
consequences = ['Environmental impact of hydraulic fracturing', 'Environmental impact of hydraulic fracturing',
                'Climate change denial', 'Climate change denial']
problems = ['Methane emissions', 'Methane emissions', 'Methane emissions', 'Climate change denial']
solutions = ['Renewable natural gas', 'Biogas', 'Biofuel', 'Scientific consensus on climate change']

In [18]:
cc_relations = [(causes[i], consequences[i]) for i in range(len(causes))]
# cc_relations.extend([(consequences[i], causes[i]) for i in range(len(causes))])
ps_relations = [(problems[i], solutions[i]) for i in range(len(problems))]
# ps_relations.extend([(problems[i], solutions[i]) for i in range(len(problems))])

In [25]:
len(cc_relations)

4

In [26]:
len(ps_relations)

4

In [19]:
data = {'nodes': [], 'links': []}

In [20]:
for i in range(len(df_lda)):
    title = df_lda['Title'].iloc[i]
    data['nodes'].append({'id': str(i), 'label': title})
    
    lda = df_lda['Sims'].iloc[i]
    cat = df_category['Jaccard'].iloc[i]
    links = df_links['Jaccard'].iloc[i]
    
    for j in range(len(df_lda)):
        if i != j and j > i:
            title2 = df_lda['Title'].iloc[j]
            
            if (title, title2) in cc_relations or (title2, title) in cc_relations:
                link_type = 'cc'
            elif (title, title2) in ps_relations or (title2, title) in ps_relations:
                link_type = 'ps'
            else:
                link_type = 'na'
            
            data['links'].append({
                'source': str(i),
                'target': str(j),
                'type': link_type,
                'w_lda': lda[j],
                'w_cat': cat[j],
                'w_links': links[j]
            })

In [27]:
len([link for link in data['links'] if link['type'] != 'na'])

9

In [28]:
with open("../data/data_viz.json", 'w+', encoding='utf-8') as f:
    json.dump(data, f, indent='\t')